# Optimización convexa con CVXPY

Construcción de un portafolio con restricciones personalizadas utilizando `cvxpy`.


> **Instalación**
>
> Ejecuta la siguiente celda si necesitas instalar las librerías.


In [ ]:
!pip install yfinance cvxpy


## Datos y estadísticos


In [ ]:
import yfinance as yf
import pandas as pd
import cvxpy as cp
import numpy as np

tickers = ['XLK', 'XLF', 'XLE', 'XLY', 'XLV']
prices = yf.download(tickers, start='2019-01-01', end='2023-12-31')['Adj Close']
returns = prices.pct_change().dropna()
mu = returns.mean() * 252
Sigma = returns.cov() * 252
mu, Sigma.head()


## Problema de optimización


In [ ]:
n = len(tickers)
w = cp.Variable(n)
target_return = 0.12
risk = cp.quad_form(w, Sigma.values)
objective = cp.Minimize(risk)
constraints = [
    cp.sum(w) == 1,
    w >= 0,
    mu.values @ w >= target_return,
    w[0] <= 0.4,  # limitar exposición al sector tecnológico
]
prob = cp.Problem(objective, constraints)
prob.solve()
{ticker: weight for ticker, weight in zip(tickers, w.value)}


El resultado entrega las ponderaciones que minimizan la varianza cumpliendo las restricciones definidas.
